In [1]:
from deap import base, creator, tools
import random
import numpy as np
import math

In [2]:
n_cities = 7
n_paths = 5

In [3]:
# generate a matrix of instances
# always start from city 0 and return to it
initial_population = np.outer(np.ones(n_paths), np.arange(1,n_cities))
initial_population.astype(int)
# permute the instances to start
initial_population =np.apply_along_axis(np.random.permutation, 1,initial_population )
initial_population

array([[3., 1., 6., 5., 2., 4.],
       [3., 5., 4., 6., 1., 2.],
       [2., 1., 5., 4., 6., 3.],
       [3., 4., 5., 1., 6., 2.],
       [4., 5., 3., 6., 2., 1.]])

In [4]:
# generate position of the cities
x_coord = np.random.uniform(0,100,n_cities)
y_coord = np.random.uniform(10,30,n_cities)
print(x_coord)
print(y_coord)

[38.11567391 53.26326768 15.06103601 46.17594835 68.40836695 20.93845485
 42.95670411]
[19.08660874 11.22029834 25.64620278 29.15054996 13.01477592 18.08412583
 20.93747047]


In [5]:
#fitness function
def distance (x1, y1, x2, y2):
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

In [6]:
# generate the matrix of the distances
#preallocat matrix
distances = np.zeros((n_cities,n_cities))
for i in range(n_cities):
    for j in range(n_cities):
        distances[i,j] = distance(x_coord[i], y_coord[i], x_coord[j], y_coord[j])
distances

array([[ 0.        , 17.06834603, 23.96966004, 12.89383329, 30.89521655,
        17.20644724,  5.18278521],
       [17.06834603,  0.        , 40.83524487, 19.28014571, 15.25103871,
        33.04550881, 14.1650516 ],
       [23.96966004, 40.83524487,  0.        , 31.31163073, 54.8223555 ,
         9.57752891, 28.29028912],
       [12.89383329, 19.28014571, 31.31163073,  0.        , 27.47077795,
        27.55715553,  8.82146293],
       [30.89521655, 15.25103871, 54.8223555 , 27.47077795,  0.        ,
        47.73982471, 26.65626062],
       [17.20644724, 33.04550881,  9.57752891, 27.55715553, 47.73982471,
         0.        , 22.20236195],
       [ 5.18278521, 14.1650516 , 28.29028912,  8.82146293, 26.65626062,
        22.20236195,  0.        ]])

In [7]:
# always start from the city 0 and return to it.
def compute_fitness(population):
    fit_values = np.ones(n_paths)
    pos = 0
    for path in population:
        l = 0
        for i in range(n_cities-2):
            l = l + distances[int(path[i]),int(path[i+1])]
        l = l + distances[0,int(path[0])]
        l = l + distances[int(path[n_cities-2]),0]
        fit_values[pos] = l  
        pos = pos + 1
    return np.argsort(-fit_values)

In [8]:
compute_fitness(initial_population)

array([4, 2, 1, 3, 0])

In [9]:
#selection function
# ordina i cammini in base alla fitnes
# genera un numero x tra 0 e n_paths
# scegli i primi x cammini
def selection(population):
    indices = compute_fitness(population)
    p = np.random.randint(0,n_paths,1)
    i = 0
    fittest_parents = np.outer(np.ones(n_paths), np.arange(1,n_cities))
    while(i < p):
        fittest_parents[i] = population[indices[i]]
        i = i+ 1
    return fittest_parents

In [10]:
selection(initial_population)

array([[1., 2., 3., 4., 5., 6.],
       [1., 2., 3., 4., 5., 6.],
       [1., 2., 3., 4., 5., 6.],
       [1., 2., 3., 4., 5., 6.],
       [1., 2., 3., 4., 5., 6.]])

In [11]:
def build_children(v1,v2):
    intersection = np.intersect1d(v1,v2)
    print(intersection)
    all_cities = np.arange(1,n_cities)
    # se v1 e v2 non hanno citta in comune
    if (len(intersection) > 0):
        diff = np.setdiff1d(v1, intersection)
        v1 = [o for o in v1 if o in diff]
    excluded = np.concatenate((v1,v2))
    third = np.setdiff1d(all_cities, excluded)
    return np.concatenate((v1,third,v2))

In [14]:
build_children([2,1,4,5],[3,4,5])

[4 5]


array([2, 1, 6, 3, 4, 5])

In [34]:
# dati i primi x migliori genitori
# per ogni cammino (n_paths)
# scelgo a caso due genitori
# scelgo a caso due punti in cui spezzare
# devo assicurarmi che sia un cammino corretto
def crossbreeding(population):
    parents = selection(population)
    n_parents = len(parents)
    for i in range(n_paths-1):
        parent1 = np.random.randint(0,n_parents,1)
        parent2 = np.random.randint(0,n_parents,1)
        if (parent1 != parent2) :
            pos1 = np.random.randint(0,n_cities,1)[0]
            pos2 = np.random.randint(0,n_cities,1)[0]
            parent1 = parents[parent1][0]
            parent2 = parents[parent2][0]
            population[i] = build_children(parent1[0:pos1], parent2[pos2:])
            population[i+1] = build_children(parent1[pos2:], parent2[0:pos1])
        else:
            population[i] = parent1
            population[i+1] = parent2
    return population     

In [35]:
crossbreeding(initial_population)

[3. 4. 5.]
[1. 2.]
[1. 2. 3. 4. 5.]
[1. 2. 3. 4. 5.]


array([[0., 0., 0., 0., 0., 0.],
       [2., 2., 2., 2., 2., 2.],
       [1., 2., 3., 4., 5., 6.],
       [1., 2., 3., 4., 5., 6.],
       [6., 1., 2., 3., 4., 5.]])

In [ ]:
#mutate